# Computer Assignment #2
## Ali Hamzehpour 810100129
## Part One: Genetic Algorithm (Buying stocks using genetic algorithm)
### Problem Description
We are given data of some company stocks' prices from 2020 to 2022. The data includes the ```return``` of the stock and its ```risk```. ```return``` indicates that how much the price of the stock has increased in that time and ```risk``` is the maximum fall in the price of the stock. We have to give each companies' stock a coefficient (from 0 to 1) that shows how much we want to invest in that stock. We have some limitations: we have to have shares in at least 30 companies, sum of the risk should be less than 60% and sum of the return should be more than 1000%.

First we load the data and check its general features.

In [1]:
import pandas as pd
data =  pd.read_csv("sample.csv")

In [2]:
data.head(10)

,Unnamed: 0,ticker,risk,return
0,0,LMB,0.665555,2.459677
1,1,GECCM,0.011733,0.202107
2,2,OXBR,0.810532,2.040541
3,3,LIVE,0.630844,3.729542
4,4,NVEE,0.330513,1.610970
5,5,ANAB,0.468864,0.520464
6,6,FNK,0.430869,0.331228
7,7,ADRA,0.026280,0.089737
8,8,HDSN,0.262239,9.525773
9,9,GNPX,0.775387,3.323529


In [3]:
data.describe()

,Unnamed: 0,risk,return
count,400.000000,400.000000,400.000000
mean,199.500000,0.460890,0.754245
std,115.614301,0.206229,1.364516
min,0.000000,0.000000,0.000565
25%,99.750000,0.322457,0.127440
50%,199.500000,0.420737,0.317390
75%,299.250000,0.607160,0.817212
max,399.000000,0.959499,16.480000


We have some constants in our problem that we should find their best value for this problem. so I defined a ```Constants``` dataclass to pass it to my functions and easily run the algorithm with different constant values.

In [4]:
from dataclasses import dataclass
import numpy as np

NUM_OF_STOCKS = 400

@dataclass
class Constants:
    max_generations: int
    population_length: int
    p_crossover: float
    p_mutation: float
    carry_size: int


I defined the gene as a number from 0 to 1 which shows a company's coefficient in our data. So a chromosome will be a list with length of 400 (number of companies) which each element in the list is the coefficient of a company's stock.

To make a random chromosome I made a list with random values from 0 to 1 and the normalize it. In normalization, at first I also implemented a threshold in which I set values that are too small(less than sum / (number of companies * 10)) as 0, then I simply divided each value in the list by the sum of the list.

In [46]:
import random
import warnings

def normalize_chromosome(chromosome):
    chromosome[chromosome < chromosome.sum()/(NUM_OF_STOCKS*10)] = 0
    return chromosome / chromosome.sum()

def gen_random_chromosome():
    chromosome = np.random.rand(NUM_OF_STOCKS)
    return normalize_chromosome(chromosome).reshape((1,400))
    

To make an initial population, I simply generate random chromosomes with the size of the population. 

In [11]:
def gen_initial_population(population_length):
    return np.array([gen_random_chromosome() for i in range(population_length)]).reshape((population_length, NUM_OF_STOCKS))

I define fitness function as below:
$$ extraPunishment(chromosome) = max(risk_i - 0.6, 0) * 5 $$
$$ fitness(chromosome) = max(0, sum(return_i - risk_i - extraPunishment(chromosome)) * c_i) $$ 
which $c_i$ is the coefficient of each company's stock. <br>
the part $return_i - risk_i$ is obvious and the part $(risk_i - 0.6) * 5$ is to put extra punishment for stocks that their risk are more than 0.6. The maximum part is to avoid having negative fitnesses.

In [12]:
def calc_fitness(chromosome):
   return_fitness = data["return"].to_numpy().reshape((1,NUM_OF_STOCKS))
   risk_fitness = data["risk"].to_numpy().reshape((1,NUM_OF_STOCKS))  
   additional_risk = np.maximum((data["risk"] - 0.6).to_numpy().reshape((1,NUM_OF_STOCKS)), np.zeros(NUM_OF_STOCKS).reshape(1, NUM_OF_STOCKS))
   total_return = np.multiply(chromosome, return_fitness).sum()
   total_risk = np.multiply(chromosome, risk_fitness + additional_risk * 5).sum()
   fitness = (total_return - total_risk).sum()
   return max(fitness, 0)

In [13]:
calc_fitness(gen_random_chromosome())

0.07855732667841564

For crossover I implemented two methods: one-point crossover and uniform crossover. In one-point I choose a random index from 0 to 400 and then slice parents at that point. Then children will be made by changing parents' parts from that point. In the end, I normalize children's chromosomes. In uniform crossover for each gene we decide to give which parent's gene to the first child and the second child will be the inverse of the first child.

In [35]:
def crossover_one_point(parents, p_crossover):
    if np.random.random() > p_crossover:
        return parents
    par1, par2 = parents
    non_zero_sum = False
    while not non_zero_sum:
        crossover_point = np.random.randint(NUM_OF_STOCKS)
        child1 = np.concatenate((par1[crossover_point:], par2[:crossover_point]))
        child2 = np.concatenate((par2[crossover_point:], par1[:crossover_point]))
        if child1.sum() > 0 and child2.sum() > 0:
            non_zero_sum = True
    return normalize_chromosome(child1), normalize_chromosome(child2)

In [15]:
def uniform_crossover(parents, p_crossover):
    if np.random.random() > p_crossover:
        return parents
    par1, par2 = parents
    child1, child2 = np.empty(shape = (NUM_OF_STOCKS)), np.empty(shape = (NUM_OF_STOCKS))
    for i in range(len(par1)):
        if np.random.bit_generator.randbits(1):
            child1[i] = par1[i]
            child2[i] = par2[i]
        else:
            child1[i] = par2[i]
            child2[i] = par1[i]
    return normalize_chromosome(child1), normalize_chromosome(child2)

In mutation I choose a random gene and set its value completely random and then I normalize the new chromsome.

In [175]:
def mutate(chromosome, p_mutation):
    if np.random.random() > p_mutation:
        return chromosome
    index = np.random.randint(NUM_OF_STOCKS)
    chromosome[index] = np.random.random()

    return normalize_chromosome(chromosome)

For making mating pool I use rank-based selection. In the main algorithm function(```find_optimal_solutions```) for each generation, I calculate fitnesses and then check if we have an answer or not, then I choose some of the best chromosomes to directly carry them to the next generation then I make a new mating pool and run crossover on the mating pool. For crossover I used one-point crossover(Uniform is faster though.) In the end I run mutation on the new population.

In [16]:
return_col = data["return"].to_numpy().reshape((1,NUM_OF_STOCKS))
risk_col = data["risk"].to_numpy().reshape((1,NUM_OF_STOCKS))

In [155]:
def can_be_solution(chromosome):
    used_stocks = np.count_nonzero(chromosome)
    total_risk = np.multiply(chromosome, risk_col).sum()
    total_return = np.multiply(chromosome, return_col).sum()
    return (used_stocks > 30 and total_return > 10 and total_risk < 0.6)

def make_mating_pool(population, population_length, mating_pool_length, prob):
    prob = prob / prob.sum()
    indices = np.random.choice(population_length, mating_pool_length, p = prob)
    unique, counts = np.unique(indices, return_counts=True)
    return population[indices, :].reshape((-1, 2, NUM_OF_STOCKS))
       
def find_optimal_solution(constants):
    population = gen_initial_population(constants.population_length)
    generation_num = 0
    while generation_num < constants.max_generations:
        fitnesses = np.apply_along_axis(calc_fitness, axis = 1, arr = population)
        sorted_indices = np.argsort(fitnesses)
        population = population[sorted_indices]
        is_solution = np.apply_along_axis(can_be_solution, axis = 1, arr = population)
        if np.count_nonzero(is_solution):
            return population[np.nonzero(is_solution)[0],:], generation_num
        #print(f"{generation_num}:")
        #print(max(fitnesses), calc_fitness(population[-1]), np.count_nonzero(population[-1]))
        weights = np.array([i for i in range(1, constants.population_length + 1)])
        carry_population = population[-constants.carry_size:]
        mating_pool = make_mating_pool(population, constants.population_length, constants.population_length - constants.carry_size, weights)
        crossover_population = np.array([crossover_one_point(parents, constants.p_crossover) for parents in mating_pool]).reshape((-1, NUM_OF_STOCKS))
        crossover_population = np.apply_along_axis(lambda x: mutate(x, constants.p_mutation), axis = 1, arr = crossover_population).reshape((-1, NUM_OF_STOCKS))
        population = np.concatenate((crossover_population, carry_population))
        generation_num += 1
    return None, constants.max_generations


### Testing

I wrote a wrapper function for ```find_optimal_solution``` to run the function multiple times and calculates the average generations and time spent.

In [199]:
import time
def run_genetic(constants):
    print(f" constants: max_generations: {constants.max_generations} population_length: {constants.population_length} p_crossover: {constants.p_crossover} p_mutation: {constants.p_mutation} carry_size: {constants.carry_size}")
    sum_num_of_generations = 0
    sum_time = 0
    REPEAT = 3
    found_answer = 0
    for i in range(REPEAT):
        t0 = time.time()
        answer, num_of_generations = find_optimal_solution(constants)
        if type(answer) == type(None):
            print("Couldn't find answer")
            return 
        t1 = time.time()
        sum_time += (t1 - t0)
        sum_num_of_generations += num_of_generations
    print(f"On average took {sum_num_of_generations / REPEAT} generations and took {sum_time / REPEAT} seconds.")

I run the algorithm with differnet constants:

In [178]:
constants = Constants(1000, 700, 0.6, 0.1, 70)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 700 p_crossover: 0.6 p_mutation: 0.1 carry_size: 70
On average took 76.0 generations and took 4.807551781336467 seconds.


In [205]:
constants = Constants(1000, 700, 0.4, 0.1, 70)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 700 p_crossover: 0.4 p_mutation: 0.1 carry_size: 70
On average took 63.0 generations and took 3.888690710067749 seconds.


In [204]:
constants = Constants(1000, 700, 0.4, 0.01, 70)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 700 p_crossover: 0.4 p_mutation: 0.01 carry_size: 70
On average took 389.0 generations and took 22.191707611083984 seconds.


In [203]:
constants = Constants(1000, 3000, 0.6, 0.1, 300)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 3000 p_crossover: 0.6 p_mutation: 0.1 carry_size: 300
On average took 43.666666666666664 generations and took 11.503020286560059 seconds.


In [201]:
constants = Constants(1000, 700, 0, 0.1, 100)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 700 p_crossover: 0 p_mutation: 0.1 carry_size: 100
On average took 29.0 generations and took 1.7685893376668294 seconds.


In [202]:
constants = Constants(1000, 700, 0.7, 0, 100)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 700 p_crossover: 0.7 p_mutation: 0 carry_size: 100
Couldn't find answer


In [184]:
constants = Constants(1000, 300, 0.7, 0.1, 30)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 300 p_crossover: 0.7 p_mutation: 0.1 carry_size: 30
On average took 319.0 generations and took 8.319308201471964 seconds.


In [196]:
constants = Constants(1000, 300, 0.4, 0.1, 70)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 300 p_crossover: 0.4 p_mutation: 0.1 carry_size: 70
On average took 70.66666666666667 generations and took 1.8000747362772624 seconds.


In [186]:
constants = Constants(1000, 300, 0.4, 0.01, 30)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 300 p_crossover: 0.4 p_mutation: 0.01 carry_size: 30
On average took 380.6666666666667 generations and took 9.407411257425943 seconds.


In [191]:
constants = Constants(1000, 10 , 0.7, 0.1, 2)
run_genetic(constants)

 constants: max_generations: 1000 population_length: 10 p_crossover: 0.7 p_mutation: 0.1 carry_size: 2
Couldn't find answer


### Result
I chose the one with population size of 700 and p_crossover = 0.6 and p_mutation = 0.1 to find my final results

In [188]:
answer, _ = find_optimal_solution(Constants(1000, 700, 0.6, 0.1, 70))
print(answer, _)
np.count_nonzero(answer)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.56782872e-04
  2.69760610e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.60532711e-04 2.61055542e-04
  0.00000000e+00 4.54274941e-04 4.67776586e-04 0.00000000e+00
  4.32856325e-04 0.00000000e+00 5.63921446e-04 2.63250711e-04
  0.00000000e+00 3.68999325e-04 0.00000000e+00 5.40017977e-04
  0.00000000e+00 3.67015612e-04 3.84083738e-04 0.00000000e+00
  2.97034599e-04 4.36579608e-04 0.00000000e+00 5.58449517e-04
  3.34629274e-04 2.92546772e-04 0.00000000e+00 4.67630397e-04
  4.37327078e-04 4.74030200e-04 4.86380070e-04 3.76291673e-04
  4.37074762e-04 2.96102305e-04 5.72130532e-04 0.00000000e+00
  5.68591450e-04 0.00000000e+00 4.24941174e-04 0.00000000e+00
  3.08395616e-04 0.00000000e+00 5.61874889e-04 5.83041526e-04
  3.5225

225

In the end I saved the answer in ```my_answer.csv``` file.

In [189]:
answer = answer.reshape(NUM_OF_STOCKS).tolist()
data["coeffs"] = answer
data.to_csv('my_answer.csv', index=False)

### Questions
1. *What happens if our population is too big or small?* If we have a big population, It may help us in terms of finding the optimal solution in less generations because we have more diversity but the problem is it will take more time and memory. If the population is too small we have less diversity and we may never find an optimal answer.

2. *What happens if the population increases after each generations?* If the population grows with each generation, the diversity increases and it may help us but again it will take more time and memory. Also we keep the population the same to make our chromosomes converge to optimal answer and if we increase the population, it may harm the convergence.

3. *What are the effects of crossover and mutation and what happens if we use only one of them?* Crossover makes better chromosomes by combining good chromosomes but mutation is mostly used to make better chromosomes by directly changing the genes of one chromosome and helps us escape a local extremum. For example if we check the best fitness in the population of each generation, we can see that it converges to a number and then a mutation causes it to diverge from that number and suddenly increase. So we can't solve the problem by only using crossover.(uniform crossover may reach the answer but it will take too many generations) and in general only using mutation won't lead us to an optimal answer because it doesn't have the effect of crossover, however in this case for some constants, only using mutation leads to answer.

4. *What are some ways to find the answer faster?* We can play with parameters as I did in testing part to find the best values to find the answer as fast as possible.

5. *How can we solve the problem of chromosomes getting stuck after some generations?* As I said in question 3, mutation usually solves ths problem. we could also use multi-starting and run the algorithm with different initial populations.

6. *What can we do to stop the algorithm if the problem doesn't have any solutions?* We can define a maximum value for the number of the generations (as I did) and stop the algorithm if it reaches that limit. Then we can return the chromosome with the best fitness in out population.

## Part Two: Game (Minimax For Othello)
### Problem description:
We are given a code for a game called [Othello](https://en.wikipedia.org/wiki/Reversi#Othello) and we are asked to complete one of the AI agents of the game. The other agent chooses its moves randomly. We have to use minimax algorithm with alpha-beta pruning.

I changed the code design. We have four classes in the code:
* ```Board```: stores the board of the game and has the functions related to the board like changing the board when a move is made or giving all valid moves and etc.
* ```AI```: contains the minimax and other functions related to our AI agent.
* ```OthelloUI```: It can be used to graphically show the game. this class is not included in the report but can be found in ```game.py``` file.
* ```Othello```: instantiate all of other objects and handles the whole game.


In [106]:
import random
import time
import math
from copy import deepcopy

### Board


board is simply a nested list to represent the map of the game. I added some functions to the original code:
* ```get_map``` makes a deep copy of map and returns it. It's used by Othello to pass it to the UI class.
* ```count_coins``` gets a player and count its coins in the game.
* ```count_corner_coins``` gets a player and count how many coins in the 4 corners of the map it has. Corners are important in othello and I used it in the AI's heuristic function.

Other functions are the same as the ones in the original code and I simply moved them to this class.

In [116]:
class Board:
    def __init__(self, size):
        self.size = size
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
    
    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)
    
    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player
    
    def get_winner(self):
        white_count = self.count_coins(1)
        black_count = self.count_coins(-1)
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0
                        
    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0
    
    def get_map(self):
        return deepcopy(self.board)
        
    def count_coins(self, player):
        return sum([row.count(player) for row in self.board])
    
    def count_corner_coins(self, player):
        corner_coins = 0
        for i in [0, self.size - 1]:
            for j in [0, self.size - 1]:
                if self.board[i][j] == player:
                    corner_coins += 1
        return corner_coins

### Othello

Othello's functions are almost the same as the original code. The difference is it has ```ai_agent``` object which is an instance of ```AI``` class and is used to get our move in the game. some of the functions are moved to ```Board``` and I also deleted the UI part in the report. (But UI can still be seen in ```game.py``` file).

In [103]:
class Othello:
    def __init__(self, size, minimax_depth=1, prune=True):
        self.size = size
        self.board = Board(self.size) 
        self.current_turn = random.choice([1, -1])
        self.ai_agent = AI(1, minimax_depth, prune)
        
    def get_cpu_move(self):
        moves = self.board.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    def get_human_move(self):
        return self.ai_agent.get_move(deepcopy(self.board))
        
    def play(self):
        winner = None
        while not self.board.terminal_test():
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.board.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.board.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
        winner = self.board.get_winner()
        return winner, self.ai_agent.visited_nodes

### AI


#### Minimax
For this part I used the minimax algorithm. The general idea in the algorithm is to assume that our enemy is smart and chooses the best move. So we search the game tree and for the enemy's turn, we choose the move that's the worst for us(Min) and for our turn we choose the move that's the best for us (Max). The problem is we can't search the game's tree until the end (because of memory and time limitations) so we should have a maximum depth. When we reach the maximum depth, we should evaluate that current state with an appropriate function and give it a value.

### Evaluation
I defined two heuristic functions and give each of them a weight for the main evaluation function:
* ```calc_coins_heuristic```: It compares the number of the coins that each player has and is defined as below:
$$ h_1 = \frac{coins_{player} - coins_{enemy}}{coins_{player} + coins_{enemy}}$$
* ```calc_corner_heuristic```: It compares the number of coins in the corner of the map that each player has. Corners in othello are so important because you can't change a coin which is in the corner and you can change a lot of coins using a corner coin.
$$ h_2 = \frac{CornerCoins_{player} - CornerCoins_{enemy}}{CornerCoins_{player} + CornerCoins_{enemy}}$$

I also tried to define another heuristic which checked how many moves each player has in the current state but because it made the algorithm too slow, I didn't include it in the main evaluation function.

The evaluation function is defined as below:
$$ Evaluate(State) = 5 * h_1 + 2 * h_2 $$

The weights have been found by testing the algorithm multiple times.

The evaluation mentioned is for when our state is not the end of the game. When algorithm reaches a state that is the end of the game, it simply calculates the winner and if we are the winner it gives a very high value to the state. if we are the loser, it gives a very low value otherwise it gives 0 to that state.

#### Pruning
There is a trick to make our algorithm better. It's called Alpha-beta Pruning. The idea is when are in a max layer, if we find a value that's larger than another value in other branches from the upper layer(which is a Min layer) We can prune the rest of the layer because the value of that layer definitely will not be chosen by the upper min layer. This is called Beta pruning. Alpha pruning is the same but this time we prune a min layer. Alpha-beta pruning is implemented and I tested the algorithm with pruning and without it to show its effect.

In [107]:
class AI:
    def __init__(self, player, minimax_depth, prune):
        self.player = player
        self.minimax_depth = minimax_depth
        self.prune = prune
        self.visited_nodes = 0
        
    def end_match_evaluate(self, board):
        winner = board.get_winner()
        if winner == self.player:
            return 1000
        elif winner == -self.player:
            return -1000
        else:
            return 0
        
    def evaluate(self, board):            
        return  5 * self.calc_corner_heuristic(board) + 2 * self.calc_coins_heuristic(board) 
    
    def calc_mobility_heuristic(self, board):
        player_moves = len(board.get_valid_moves(self.player))
        enemy_moves = len(board.get_valid_moves(-self.player))
        if enemy_moves + player_moves == 0:
            return 0
        return (player_moves - enemy_moves) / (player_moves + enemy_moves)
    
    def calc_coins_heuristic(self, board):
        player_coins = board.count_coins(self.player)
        enemy_coins = board.count_coins(-self.player)
        return (player_coins - enemy_coins) / (player_coins + enemy_coins)
    
    def calc_corner_heuristic(self, board):
        player_corners = board.count_corner_coins(self.player)
        enemy_corners = board.count_corner_coins(-self.player)
        if enemy_corners + player_corners == 0:
            return 0
        return (player_corners - enemy_corners) / (player_corners + enemy_corners)
        
    def max_value(self, cur_board, cur_depth = 0, alpha = -math.inf, beta = math.inf, visited_nodes = 1, prev_no_move = 0):
        if cur_depth == self.minimax_depth:
            return None, self.evaluate(cur_board), visited_nodes
        valid_moves = cur_board.get_valid_moves(self.player)
        if len(valid_moves) == 0:
            if prev_no_move:
                return None, self.end_match_evaluate(cur_board), visited_nodes
            return self.min_value(cur_board, cur_depth + 1, alpha, beta, visited_nodes + 1, 1)
        best_move = None
        best_point = -math.inf
        for move in valid_moves:
            new_board = deepcopy(cur_board)
            new_board.make_move(self.player, move)
            next_move, point, visited_nodes = self.min_value(new_board, cur_depth + 1, alpha, beta, visited_nodes + 1)
            if best_point < point:
                best_point = point
                best_move = move
            if self.prune:
                if best_point >= beta:
                    return best_move, best_point, visited_nodes
                alpha = max(alpha, best_point) 
        return best_move, best_point, visited_nodes
            
    def min_value(self, cur_board, cur_depth = 0, alpha = -math.inf, beta = math.inf, visited_nodes = 1, prev_no_move = 0):
        if cur_depth == self.minimax_depth:
            return None, self.evaluate(cur_board), visited_nodes
        valid_moves = cur_board.get_valid_moves(-self.player)
        if len(valid_moves) == 0:
            if prev_no_move:
                return None, self.end_match_evaluate(cur_board), visited_nodes
            return self.max_value(cur_board, cur_depth + 1, alpha, beta, visited_nodes + 1, 1)
        best_move = None
        best_point = math.inf
        for move in valid_moves:
            new_board = deepcopy(cur_board)
            new_board.make_move(-self.player, move)
            next_move, point, visited_nodes = self.max_value(new_board, cur_depth + 1, alpha, beta, visited_nodes + 1)
            if best_point > point:
                best_point = point
                best_move = move
            if self.prune:
                if best_point <= alpha:
                    return best_move, best_point, visited_nodes
                beta = min(beta, best_point) 
        return best_move, best_point, visited_nodes
        
    def get_move(self, cur_board):
        best_move, point, visited_nodes = self.max_value(cur_board)
        self.visited_nodes += visited_nodes
        return best_move

### Testing

I wrote a wrapper function called ```test``` to play the game for 150 times with given flags and prints its information:

In [108]:
import time 
BOARD_SIZE = 6 
REPEAT = 150

def test(minimax_depth, prune):
    wins = 0
    sum_visited_nodes = 0
    time_spent = 0
    for i in range(REPEAT):
        game = Othello(BOARD_SIZE, minimax_depth, prune)
        t0 = time.time()
        result, visited_nodes = game.play()
        t1 = time.time()
        time_spent += (t1 - t0)
        sum_visited_nodes += visited_nodes
        if result == 1:
            wins += 1
    print(f"with maximum depth:{minimax_depth} and pruning:{prune}")
    print(f"wins:{wins * 100 / (REPEAT):.3f}%")
    print(f"mean visited moves:{sum_visited_nodes / REPEAT:.3f} per game")
    print(f"mean time spent:{time_spent / REPEAT:.3f} per game")

In [109]:
test(minimax_depth = 1 , prune = 0)

with maximum depth:1 and pruning:0
wins:81.333%
mean visited moves:96.573 per game
mean time spent:0.005 per game


In [110]:
test(minimax_depth = 3 , prune = 0)

with maximum depth:3 and pruning:0
wins:98.000%
mean visited moves:3505.613 per game
mean time spent:0.126 per game


In [115]:
test(minimax_depth = 5 , prune = 0)

with maximum depth:5 and pruning:0
wins:99.333%
mean visited moves:133706.160 per game
mean time spent:4.358 per game


In [111]:
test(minimax_depth = 1 , prune = 1)

with maximum depth:1 and pruning:1
wins:75.333%
mean visited moves:95.507 per game
mean time spent:0.006 per game


In [112]:
test(minimax_depth = 3 , prune = 1)

with maximum depth:3 and pruning:1
wins:98.667%
mean visited moves:1559.707 per game
mean time spent:0.061 per game


In [113]:
test(minimax_depth = 5 , prune = 1)

with maximum depth:5 and pruning:1
wins:100.000%
mean visited moves:18697.900 per game
mean time spent:0.765 per game


In [114]:
test(minimax_depth = 7 , prune = 1)

with maximum depth:7 and pruning:1
wins:100.000%
mean visited moves:211211.580 per game
mean time spent:8.455 per game


### Test Results

| Maximum Depth | Pruning | Win Percentage | Visited Node Per Game | Time Spent Per Game|
| :-----------: | :-----: | :------------: | :-------------------: | :----------------: |
| 1 | False     | 81.333% | 96.573         |         0.005         
| 3 | False     | 98.000% | 133706.160           |         0.006       
| 5 | False     | 99.333% | 0.600          |         4.358        
| 1 | True      | 75.333% | 95.507          |         0.006        
| 3 | True      | 98.667% | 1559.707          |         0.061        
| 5 | True      | 100% | 18697.900         |         0.765         
| 7 | True      | 100% | 211211.580          |         8.455        



### Questions

1. *What were the attributes considered in heuristic function and Why?* It was explained in details in Evaluation part.In general two factors was used to evaluate a state. Firstly the coins in the corners of map because they can never change and also have high effect on changing many coins. Secondly the total number of coins for each player which is an obvious thing to consider. I also wanted to add one more factors to the evaluation function but it needed more complexity and made algorithm slow. That was to check how many moves each player has. Because it's better to reach a state that our opponent is limited or even has no moves!

2. *What is the relation between the minimax depth and calculated parameters in the test part?* If we increase the minimax depth, We'll have more chance to win the game because we visit more states in the game and can predict the game more precisely. The problem is we'll need more time and memory so we should pay attention to this trade-off.

3. *Can we choose an order in visiting nodes to maximize pruning?* We can order our moves in a way to increase pruning. We have to check the better moves first. For example first we check the moves in which a corner will be capture because these moves will probably have better/worse values compared to other moves. We also can save the states that we have seen in a hash table with their value so if we reach one of them from another path, we can compare its value to alpha/beta and if pruning is not possible we check other moves in that layer. These were the ideas to improve pruning but I think in general there is no guarantee that we can always **maximize** the pruning. 

4. *Explain Branching Factor and tell how it changes as game grows?* Branching Factor is the number of children each node has in the graph. I think the growth of branching factor depends on the players' moves but in general it increases while the game progresses and at the end of the game it starts to decrease because there will be less possible moves.

5. *Explain why pruning doesn't decrease the precision of algorithm?* Because for example when we are in a max layer and find a move with value more than Beta, it means that there is a min layer in our ancestors in which there's another branch with value less than our current move's value. So this move's value won't be chosen by that min layer and since we're currently in a max layer we will be able to choose only moves with more values than this current value so those values won't be chosen by that upper min layer neither and we can ignore them and don't check them. This won't change our result.

6. *Why when our opponent is playing randomly, it's not the best choice to play with minimax algorithm? What else can we use?* Because in minimax we assume that our opponent is completely intelligent and make the best choices for each move but for example in this project our opponent was playing randomly and didn't choose he best option for each move. suppose We have a move that leads to 2 possible moves for our opponent and Both of those moves will reach a state with value 10. We have another move that leads to 2 possible moves for our opponent and one of them will reach a state with value 1000 and the other will reach a state with value 9. In minimax we choose the first move because if we choose the second move, an intelligent opponent will take us to a state with value 9 but if we know that our opponent is working randomly we can take a risk and choose the second move. We may be lucky and get 1000 or even we get unlucky we will get 9 which is not much lower that 10! So we can choose expectimax algorithm instead of minimax. in expectimax instead of finding the minimum in the opponent's layer, we calculate the expected value of all moves. Each move has a probability and its value will be multiplied by its probability and sum of these will be the value of the layer.

### Sources I used for the project:
* [A site about Othello and its AI](http://radagast.se//othello/index.html)
* [An Analysis of Heuristics in Othello](https://courses.cs.washington.edu/courses/cse573/04au/Project/mini1/RUSSIA/Final_Paper.pdf)
* [Minimax/ Alpha beta pruning Move Ordering](https://stackoverflow.com/questions/8906430/minimax-alpha-beta-pruning-move-ordering)